In [1]:
import datetime   #Each block will have a timestamp
import hashlib    #Hasing or encrypting a block
import json
from flask import Flask, jsonify, request
from uuid import uuid4
from urllib.parse import urlparse

 
#Part 1
#Create a blockchain
#http://127.0.0.1:5000/mine_block
#http://127.0.0.1:5000/get_chain

class Blockchain:
   
    def __init__(self):
        self.chain = [] 
        self.transactions = []
        self.create_block(proof = 1, previous_hash = '0')
        self.nodes = set()
            
    def create_block(self,proof,previous_hash):
        #Define essential data for the blocks
        block = {'index' : len(self.chain) + 1,
                 'timestamp': str(datetime.datetime.now()),
                 'proof': proof,
                 'previous_hash': previous_hash,
                 'transaction' : self.transactions}
        self.transactions = []
        self.chain.append(block)
        return block
   
    def get_previous_block(self):
        return self.chain[-1]
   
    def proof_of_work(self,previous_proof):
        new_proof = 1
        check_proof = False
        while check_proof is False:
            hash_operation = hashlib.sha256(str(new_proof**2 -
                                                previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] == '0000':
                check_proof=True
            else:
                new_proof += 1
        return new_proof
               
    def hash(self,block):
        encoded_block = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encoded_block).hexdigest()
    
    def is_chain_valid(self,chain):
       
        previous_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
           
            if block['previous_hash'] != self.hash(previous_block):
                return False
           
            previous_proof = previous_block['proof']
            new_proof = block['proof']
            hash_operation = hashlib.sha256(str(new_proof**2 -
                                                previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] != '0000':
                return False
           
            previous_block = block
            block_index += 1        
        return True
   
    #Format of transaction Sender, Receiver, Amount
    def add_transaction(self,sender,receiver,amount):
        self.transactions.append({'sender' : sender,
                                  'receiver' : receiver,
                                  'amount' : amount})
       
        previous_block = self.get_previous_block()
        return previous_block['index'] + 1
    #Create 2nd Pillar
    #Add the nodes
    def add_nodes(self,address):
        parsed_url = urlparse(address)
        self.nodes.add(parsed_url.netloc)
       

#initalize the Flask 
app = Flask(__name__)  
 
node_address = str(uuid4()).replace('-','')
 
blockchain = Blockchain()
 
@app.route('/mine_block', methods = ['GET'])
def mine_block():
   
    previous_block = blockchain.get_previous_block()
   
    previous_proof = previous_block['proof']
    proof = blockchain.proof_of_work(previous_proof)
   
    previous_hash = blockchain.hash(previous_block)
    blockchain.add_transaction (sender =node_address, receiver ='sarwan',amount=1200)
    
    block = blockchain.create_block(proof, previous_hash)
   
    response = {'message': 'Congrats you just mined a block',
                'index'  : block['index'],
                'timestamp' : block['timestamp'],
                'proof' : block['proof'],
                'previous_hash' : block['previous_hash'] ,
                'transcation': block['transaction']
                }
    return jsonify(response), 200


#Getting the chain of blocks
@app.route('/get_chain', methods = ['GET'])
def get_chain():
    response = {'chain':blockchain.chain,
                'length':len(blockchain.chain)}
   
    return jsonify(response), 200

@app.route('/is_valid', methods = ['GET'])
def is_valid():
    is_valid = blockchain.is_chain_valid(blockchain.chain)
    if is_valid:
        response = {'message' : 'The chain is valid'}
    else:
        response = {'message' : 'chain is not valid'}
    return jsonify(response), 200
 
@app.route('/add_transaction', methods = ['POST'])
def add_transaction():
    json = request.get_json()
    transaction_keys = ['sender','receiver','amount']
    if not all(key in json for key in transaction_keys):
        return 'Some values are missing',400
    index = blockchain.add_transaction(json['sender'],
                                       json['receiver'],
                                       json['amount'])
    response = {'message' : f'The transaction has been added to block {index}'}
    return jsonify(response), 201
 
    
@app.route('/connect_nodes', methods = ['POST'])
def connect_nodes():
    json = request.get_json()
    nodes = json.get('nodes')
    if nodes is None:
        return "No Nodes",400
    for node in nodes:
        blockchain.add_nodes(node)
    response = {'message':'all nodes are now connected',
                'Total no. of nodes':list(blockchain.nodes)}
    return jsonify(response), 201
   
    

In [ ]:
#run the server 
app.run(host='0.0.0.0',port=5000)



 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jul/2020 10:31:02] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2020 10:31:05] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2020 10:31:29] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2020 10:32:40] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - - [10/Jul/2020 10:33:35] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - - [10/Jul/2020 10:33:45] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2020 10:34:10] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2020 10:35:30] "GET /mine_block HTTP/1.1" 200 -
